In [40]:
import pandas as pd
from math import sqrt

products = [
    {'demand': 1000, 'common': 100, 'specific': 10, 'unit': 50, 'holding': 0.2},
    {'demand': 300, 'common': 100, 'specific': 20, 'unit': 60, 'holding': 0.2},
    {'demand': 100, 'common': 100, 'specific': 25, 'unit': 30, 'holding': 0.2},
    {'demand': 50, 'common': 100, 'specific': 25, 'unit': 30, 'holding': 0.2}
]

def independent_sourcing(products):
    total_cost = 0
    for product in products:
        setup_cost = product['common'] + product['specific']
        holding_cost_rate = product['holding'] * product['unit']
        economic_order_quantity = sqrt((2 * product['demand'] * setup_cost) / holding_cost_rate)
        ordering_cost = (product['demand'] * setup_cost) / economic_order_quantity
        holding_cost = (holding_cost_rate * economic_order_quantity) / 2
        total_cost += ordering_cost + holding_cost
    return total_cost

independent_cost = independent_sourcing(products)
independent_cost

3073.9153138822376

In [41]:
#question 1
def joint_sourcing(products):
    C_common = products[0]['common']  
    S_total = C_common + sum(product['specific'] for product in products)  
    n = sqrt(sum(product['demand'] * product['holding'] * product['unit'] for product in products) / (2 * S_total))
    total_ordering_cost = n * S_total
    total_holding_cost = 0
    for product in products:
        Q = product['demand'] / n
        holding_cost = Q * product['holding'] * product['unit'] / 2
        total_holding_cost += holding_cost
    return total_ordering_cost + total_holding_cost
joint_cost = joint_sourcing(products)
joint_cost

2284.7319317591728

In [44]:
#question 2
import pandas as pd
products = pd.DataFrame(products)

S_common_first = products.at[0, 'common']
S_specific_first = products.at[0, 'specific']
H_holding_first = products.at[0, 'holding'] * products.at[0, 'unit']
D_demand_first = products.at[0, 'demand']

EOQ_first = sqrt((2 * D_demand_first * (S_common_first + S_specific_first)) / H_holding_first)
frequency_first = D_demand_first / EOQ_first

products.at[0, 'frequency'] = frequency_first
products.at[0, 'm_i'] = 1

products.loc[1:, 'Q'] = ((2 * products['demand'] * products['specific']) / (products['holding'] * products['unit'])).pow(0.5)
products.loc[1:, 'frequency'] = products['demand'] / products.loc[1:, 'Q']
products.loc[1:, 'm_i'] = round(frequency_first / products.loc[1:, 'frequency'])

products

,demand,common,specific,unit,holding,frequency,m_i,Q,nfreq,nQ
0,1000,100,10,50,0.2,6.741999,1.0,NaN,7.214167,138.616151
1,300,100,20,60,0.2,9.486833,1.0,31.622777,7.214167,41.584845
2,100,100,25,30,0.2,3.464102,2.0,28.867513,3.607083,27.723230
3,50,100,25,30,0.2,2.449490,3.0,20.412415,2.404722,20.792423


In [47]:
#question 3
h = products['holding']
C = products['unit']
D = products['demand']
m = products['m_i']
S = 100
s = products['specific']

n_updated = sqrt((products['holding'] * products['unit'] * products['demand'] * products['m_i']).sum() / 
                 (2 * (S + (products['specific'] / products['m_i']).sum())))

products['nfreq'] = n_updated / products['m_i']
products.loc[0, 'nfreq'] = n_updated
products['nQ'] = products['demand'] / products['nfreq']
holding_cost = (products['nQ'] * products['holding'] * products['unit'] / 2).sum()
ordering_cost = (products['demand'] * products['specific'] / products['nQ']).sum()

total_cost = holding_cost + ordering_cost + products['nfreq'].max() * S
total_cost

2176.2735734890193